Site [1] to portal do HyCOM que descreve as diversas implementações do modelo ao longo dos anos:

O acesso por THREDDS/OPeNDAP ou ftp é super lento e falha bastante.
Nossa melhor chance é pegar por HTTPS mesmo da URL em [2].
A descrição do experimento `GLBu0.08` 91.2 é: _Uniform (1/12° resolution) grid between 80.48°S and 80.48°N_

Os arquivos são nomeados no formato "`hycon_grade_experimento_data_hora_campo`", por exemplo `hycom_glb_912_2017010100_t000_uv3z.nc`. Os campos ice, ssh, sur, ts3z, uv3z são descritos em [1].


1. https://www.hycom.org/data/glbu0pt08/expt-91pt2
2. https://data.hycom.org/datasets/GLBu0.08/expt_91.2/data/hindcasts/2017

In [1]:
from pathlib import Path
import requests
from tqdm import tqdm
import lxml.html
import subprocess


def url_lister(url: str) -> list:
    urls = []
    r = requests.get(url, allow_redirects=True)
    dom = lxml.html.fromstring(r.text)
    for link in dom.xpath("//a/@href"):
        urls.append(link)
    return urls


def compress_netcdf(fname: Path):
    ret = None
    fname = fname.absolute()
    outfile = fname.with_suffix(".nc4")
    if not outfile.is_file():
        call = ["ncks", "-4", "-L 1", str(fname), str(outfile)]
        ret = subprocess.run(call, capture_output=True)
        print(ret.stdout.decode())
    if ret and ret.returncode == 0:
        print(f"Converted {fname} to {outfile}.")
    else:
        raise Exception(f"Could no compress file {fname}")


def download(url: str, outfile: Path):
    response = requests.get(url, stream=True, allow_redirects=True)
    total_size_in_bytes= int(response.headers.get("content-length", 0))
    block_size = 1024 #1 Kibibyte
    progress_bar = tqdm(total=total_size_in_bytes, unit="iB", unit_scale=True)
    with outfile.open(mode="wb") as f:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            f.write(data)
    progress_bar.close()
    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
        print(f"Something went wrong while writing {outfile}.")

In [2]:
url = "https://data.hycom.org/datasets/GLBu0.08/expt_91.2/data/hindcasts/2017"

urls = url_lister(url)

In [3]:
# filter ony the velocity files.
fnames = [fname for fname in urls if fname.endswith("t000_uv3z.nc")]

# filter for month
month = "09"
name_list = [fname for fname in fnames if fname.startswith(f"hycom_glb_912_2017{month}")]

In [ ]:
for fname in name_list:
    fname = Path(fname).absolute()
    local_fname = fname.parent.joinpath("hycom", fname.name)
    
    if not local_fname.with_suffix(".nc4").is_file():
        remote = f"{url}/{fname.name}"
        download(remote, local_fname)
        print(f"Downloading file from {remote}.")
        compress_netcdf(local_fname)
        local_fname.unlink()
    else:
        print(f"File {local_fname} already exits.")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.44G/1.44G [03:19<00:00, 7.23MiB/s]



Converted /home/filipe/Dropbox/Work/JueFi/Projetos/Proj_Modelagem_Oleo/hycom/hycom_glb_912_2017090100_t000_uv3z.nc to /home/filipe/Dropbox/Work/JueFi/Projetos/Proj_Modelagem_Oleo/hycom/hycom_glb_912_2017090100_t000_uv3z.nc4.
File /home/filipe/Dropbox/Work/JueFi/Projetos/Proj_Modelagem_Oleo/hycom/hycom_glb_912_2017090100_t000_uv3z.nc already exits.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.44G/1.44G [03:15<00:00, 7.37MiB/s]



Converted /home/filipe/Dropbox/Work/JueFi/Projetos/Proj_Modelagem_Oleo/hycom/hycom_glb_912_2017090200_t000_uv3z.nc to /home/filipe/Dropbox/Work/JueFi/Projetos/Proj_Modelagem_Oleo/hycom/hycom_glb_912_2017090200_t000_uv3z.nc4.
File /home/filipe/Dropbox/Work/JueFi/Projetos/Proj_Modelagem_Oleo/hycom/hycom_glb_912_2017090200_t000_uv3z.nc already exits.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.44G/1.44G [02:43<00:00, 8.83MiB/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.44G/1.44G [02:15<00:00, 10.6MiB/s]



Converted /home/filipe/Dropbox/Work/JueFi/Projetos/Proj_Modelagem_Oleo/hycom/hycom_glb_912_2017092800_t000_uv3z.nc to /home/filipe/Dropbox/Work/JueFi/Projetos/Proj_Modelagem_Oleo/hycom/hycom_glb_912_2017092800_t000_uv3z.nc4.
File /home/filipe/Dropbox/Work/JueFi/Projetos/Proj_Modelagem_Oleo/hycom/hycom_glb_912_2017092800_t000_uv3z.nc already exits.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.44G/1.44G [03:04<00:00, 7.83MiB/s]



Converted /home/filipe/Dropbox/Work/JueFi/Projetos/Proj_Modelagem_Oleo/hycom/hycom_glb_912_2017092900_t000_uv3z.nc to /home/filipe/Dropbox/Work/JueFi/Projetos/Proj_Modelagem_Oleo/hycom/hycom_glb_912_2017092900_t000_uv3z.nc4.
File /home/filipe/Dropbox/Work/JueFi/Projetos/Proj_Modelagem_Oleo/hycom/hycom_glb_912_2017092900_t000_uv3z.nc already exits.


  9%|███████████████▉                                                                                                                                                          | 135M/1.44G [01:35<1:28:37, 246kiB/s]